### Whisper + silero vad

silero vad 모델과 배경을 제거 통해 말하는 타이밍을 더 정확하게 잡아서 text추출을 더 좋게 할 수 있게 가능하다 라는 글을 보고 작성한 note book.

### 파이프 라인 전개도

![pipeline-image](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*sUAnXkxvJYReCpwXZGOz_A.png)

### 파이프 라인 전개도를 통해 얻은 결과 (whisper-largev2 모델 사용)

1. `audio` 를 바로 `whisper` 모델에 넣어 봤을 경우
   - 인식률이 떨어지는 경우 + 조금의 할루시네이션 이 발생하긴 하였지만 어느정도의 음성은 추출됨.
   - 가끔 배경의 소리를 음성으로 인식해 음성의 시작이 아닌 배경의 시작에서 부터 시간이 잡혀있거나 그런 경우가 있음.
2. `audio UVR`처리를 한 후 `whisper` 모델에 넣어 봤을 경우
   - `whisper`모델 으로 갔을 경우 택스트 추출에 꽤나 어려움을 보임. 많은 할루시네이션 발생 같은 문자가 반복되거나, 조금씩 빠져나온 배경음을 언어로 인식하는 등 기본 모델에서 보이지 않던 문제가 발생.
3. `silero VAD`처리를 한 후 `whisper` 모델에 넣어 봤을 경우
   - 글에서 설명한 모델 `UVR-MDX-NET HQ4`을 사용 해서 했을 경우에 음성의 위치는 정확하게 추출 되지만, `min_speech_duration_ms`를 낮춰 봤음에도 짦게 말하는 부분은 추출하지 못하는 문제가 발생.
   - 음성의 위치가 정확하게 추출된 상태에서 `whisper`모델 을 사용 한 결과, 기존 `whisper`에서 생긴 문제인 배경을 음성으로 인식하는 부분이 줄어듬 단, 추출 되는 음성의 양이 `51`개에서 `48`개 까지 줄어듬. (인식 못하는 부분 있음.)
4. 파이프 라인의 전개도를 모두 따라 갔을 경우.
   - `UVR`를 통해 음성 분리를 하고 `silero VAD`를 통해 voice 를 추출한 다음, `whisper`로 음성 추출 이후 `silero VAD`의 시간을 통합했을 경우 앞 `3.`에서 언급한 인식 못하는 부분은 줄고 그 부분이 정확하게 반환되지만, `whisper`가 인식을 못해서 글이 추출 되지 않는 문제가 발생.. `whisper`의 초기 값을 수정하거나 그럴 필요가 있어보임.

### 정리

- audio 분리 전처리 과정을 통한 더 좋은 오디오 추출은 가능하였지만, 그 추출된 오디오를 `whisper`로 넘겨주는 과정에서 문제가 발생하고 있음. 이후 `whisper` 파라미터를 수정해보며 성능을 올릴 수 있어 보임..  

### 참조

- [audio 분리(전처리) 과정을 통한 더 좋은 오디오 추출 에 관한 글](https://medium.com/@developerjo0517/audio-pre-processings-for-better-results-in-the-transcription-pipeline-bab1e8f63334)


In [ ]:

from silero_vad import load_silero_vad, read_audio, get_speech_timestamps
from moviepy.audio.io.AudioFileClip import AudioFileClip
import datetime
import whisper
import uuid
import os

AUDIO_DIR_PATH = "../sample/audio"
AUDIO_FILE_PATH = f"{AUDIO_DIR_PATH}/1_uma_5_min_(Vocals).wav"

whisper_model = whisper.load_model("large-v2")

vad_model = load_silero_vad()
wav = read_audio(AUDIO_FILE_PATH)
speech_time_stamps = get_speech_timestamps(
    wav, model=vad_model, return_seconds=True, min_speech_duration_ms=100)

# 결과를 저장할 리스트
transcription_results = []


for index, time_stamp in enumerate(speech_time_stamps):
    try:
        # 고유 파일 이름 생성
        unique_filename = str(uuid.uuid4())
        temp_audio_path = f"../segments/{unique_filename}.wav"

        # 오디오 파일을 자르고 임시 파일로 저장
        audio_clip = AudioFileClip(filename=AUDIO_FILE_PATH)
        new_clip = audio_clip.subclip(time_stamp["start"], time_stamp["end"])
        new_clip.write_audiofile(temp_audio_path)

        # Whisper 모델로 자른 부분의 텍스트를 추출
        result = whisper_model.transcribe(
            temp_audio_path, language="Japanese", temperature=0.2
        )

        # 결과를 출력하고 리스트에 저장
        transcription_results.append({
            "index": index,
            "start": time_stamp["start"],
            "end": time_stamp["end"],
            "text": result['text']
        })

        print(
            f"Index {index}: Transcription: {result['text']}")

    except Exception as e:
        print(f"오류 발생 (Index {index}): {e}")

    finally:
        # 임시 파일 삭제
        if os.path.exists(temp_audio_path):
            os.remove(temp_audio_path)

# VTT 파일로 저장
vtt_file_path = f"../output/whisper_vad_{datetime.datetime.now().microsecond}.vtt"

with open(vtt_file_path, 'w', encoding='utf-8') as vtt_file:
    vtt_file.write("WEBVTT\n\n")

    for result in transcription_results:
        start_time = result["start"]
        end_time = result["end"]
        text = result["text"]

        # 시간을 VTT 포맷 (HH:MM:SS.mmm)으로 변환
        start_time_str = f"{int(start_time // 3600):02}:{int((start_time % 3600) // 60):02}:{start_time % 60:06.3f}"
        end_time_str = f"{int(end_time // 3600):02}:{int((end_time % 3600) // 60):02}:{end_time % 60:06.3f}"

        # VTT 형식으로 자막을 작성
        vtt_file.write(f"{result['index'] + 1}\n")
        vtt_file.write(f"{start_time_str} --> {end_time_str}\n")
        vtt_file.write(f"{text}\n\n")

print(f"VTT 파일이 '{vtt_file_path}'로 저장되었습니다.")

MoviePy - Writing audio in ../segments/bc6b2a2a-3da2-450e-8f82-1be2838ac6c7.wav


MoviePy - Done.


Index 0: Transcription: うまむすめ
MoviePy - Writing audio in ../segments/ab764ab4-0cf9-4089-8e67-5b39421d0d59.wav


MoviePy - Done.


Index 1: Transcription: 彼女たちは
MoviePy - Writing audio in ../segments/12e7cb70-79f0-4715-8894-92a4f1f2fc45.wav


...

Index 44: Transcription: 
MoviePy - Writing audio in ../segments/9c7d79bf-85ab-4dc5-9274-5650752e0a2e.wav


MoviePy - Done.


Index 45: Transcription: 優先から負け出すの読んだ
MoviePy - Writing audio in ../segments/519de236-a148-4a7a-92f9-d800741e8cc2.wav


MoviePy - Done.


Index 46: Transcription: クラシック3巻の夢向け大いに期待が高まります
VTT 파일이 '../output/whisper_vad_257013.vtt'로 저장되었습니다.
